In [ ]:
import difflib
import math

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import json
import os

research_dir = "./research/"

try: 
    os.makedirs(research_dir)
except: 
    pass

In [16]:

# GEMINI

def compare_strings(str1, str2):
    """Compares two strings and returns a list of tuples indicating changes.

    Args:
        str1: The original string.
        str2: The modified string.

    Returns:
        A list of tuples, each containing a range of indices and the replacement text.
    """

    matcher = difflib.SequenceMatcher(None, str1, str2)
    diffs = matcher.get_opcodes()

    changes = []
    for tag, i1, i2, j1, j2 in diffs:
        if tag == 'delete':
            changes.append((tag, (i1, i2), ""))
        elif tag == 'insert':
            changes.append((tag, (i1, i1), str2[j1:j2]))
        elif tag == 'replace':
            changes.append((tag, (i1, i2), str2[j1:j2]))

    return changes

def score_dutch_text(reference_text, generated_text,language = 'dutch'):
    """
    Scores the quality of a Dutch generated text compared to a reference text.

    Args:
        reference_text (str): The reference Dutch text.
        generated_text (str): The generated Dutch text.

    Returns:
        float: A score between 0 and 1, with 1 being a perfect match.
    """

    reference_tokens = word_tokenize(reference_text, language=language)
    generated_tokens = word_tokenize(generated_text, language=language)

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    # Calculate word-level accuracy
    correct_words = 0
    # for ref_word, gen_word in zip(reference_tokens, generated_tokens):
    #     if ref_word == gen_word:
    #         correct_words += 1
    word_accuracy = correct_words / len(reference_tokens)

    # Calculate edit distance using Difflib
    matcher = difflib.SequenceMatcher(None, reference_text, generated_text)
    ops = matcher.get_opcodes()
    edit_distance_penalty = 0
    for tag, i1, i2, j1, j2 in ops:
        if tag == 'delete' or tag == 'insert' or tag == 'replace':
            edit_distance_penalty += (i2 - i1) + (j2 - j1)
            
    average_text_length = (len(reference_text) +  len(generated_text)) / 2

    # Calculate the final score
    final_score = (abs(bleu_score - 1) * 0.5) + (word_accuracy * 0.3) + (edit_distance_penalty / len(reference_text))

    return float(final_score)

In [14]:
# OPEN AI gpt 4o
cruijff_score_inputs = [

    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schieten, nders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "enkele letterverandering",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schoten, anders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "enkele woordverandering, geen betekenisverandering",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet roeien, anders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "enkele woordverandering, wel betekenisverandering",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schoten, anders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "woordweglating",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schoten,.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "zinsdeelweglating",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "tekstweglating",
    },
]
different_score_input = [
    {
        "quote": "Ik heb een heel zwaar leven.",
        "quote_changed": "Ik heb een heel zwaar leven.",
        "person": "Brigitte Kaandorp",
        "year": 2009,
        "test": "nulmeting",
    },
    {
        "quote": "Ik geloof in God, behalve als ik vis.",
        "quote_changed": "Ik geloof in God, be",
        "person": "Herman Brood",
        "year": 1995,
        "test": "weglating aan einde",
    },
    {
        "quote": "Als het niet kan zoals het moet, dan moet het maar zoals het kan.",
        "quote_changed": "Als het niet kan zoals  het maar zoals het kan.",
        "person": "Dolf Jansen",
        "year": 2005,
        "test": " weglating in midden",
    },
    {
        "quote": "Ik heb nooit last van hoogtevrees, wel van dieptevrees.",
        "quote_changed": "Ik hebt ooit last van hoogtevrees, well vann dieptevrees.",
        "person": "Youp van 't Hek",
        "year": 1998,
        "test": "enkele letterweglating, betekenisverandering",
    },
    {
        "quote": "Ik ben niet dik, ik ben een ruimtewonder.",
        "quote_changed": "Ik bn nit dik, ik bn n ruimtwondr.",
        "person": "Brigitte Kaandorp",
        "year": 2003,
        "test": "letter e weggelaten",
    },
    {
        "quote": "Een dag niet gelachen is een dag niet geleefd.",
        "quote_changed": "Een dag niet gelachen is een dag niet geleeft.",
        "person": "Charlie Chaplin",
        "year": 1930,
        "test": "enkele letterverandering , geen betekenisverandering",
    },
    {
        "quote": "Een dag niet gelachen is een dag niet geleefd.",
        "quote_changed": "Een  niet gelachen is een dag niet geleefd.",
        "person": "Charlie Chaplin",
        "year": 1930,
        "test": "woordweglating",
    },
    {
        "quote": "Ik ben niet gek, ik ben een vliegtuig.",
        "quote_changed": "Ik ben niet , ik  een .",
        "person": "Supergrover",
        "year": 1974,
        "test": "dubbelle woordweglating",
    },
    { 
        "quote": "Ik begrijp niet waarom u hier zo negatief en vervelend over doet. (...) Laten we blij zijn met elkaar! Laten wij optimistisch zijn! Laten we zeggen: Nederland kan het weer! Die VOC-mentaliteit, over grenzen heen kijken, dynamiek! Toch?",
        "quote_changed": "Ik begrijp niet waarom u hier zo negatief en vervelend over doet. (...) Laten we blij zijn met elkaar! Laten wij optimistisch zijn! Laten we zeggen: Nederland kan het weer! Die",
        "person": "Jan-Peter Balkenende",
        "year": 2006,
        "test": "weglating einde van grotere tekst",
    },
    { 
        "quote": "Praat Nederlands met me. Even Nederlands met me. Mijn gevoel zegt mij dat wij vanavond samen kijken. Naar de Champs-Élysées en naar de Notre Dame en naar de Seine. En daarna samen op La Tour Eiffel",
        "quote_changed": "Praat Nedertands met me. Even Neterlands met me. Mijn tevoet zegt mij dat wij vanatond samet kitken. Naar de Champs-Éltsées en naar de Notre Dameten naar detSeine. En daarta samet op La Tour Etffel",
        "person": "Kenny B",
        "year": 2015,
        "test": "random lettermutaties",
    },
    {
        "quote": "Rrrrrr, hah, is gewoon Boef man. Ha, jij bent vies maar ik doe gemener. In de club, kom je moeder tegen. En ik wil snel weg want we moeten wegen. En je klant is geholpen, je moet vroeger wezen. Ik was alles kwijt, maar floes herenigd. Voor me zondes af en toe gebeden. Ik ga uit eten voor een goede prijs. Ik ben een uitgever, ze boeken mij. Van alarm voorzien aan de achterkant. Dus ze komen via voor, maar wat dacht je dan?",
        "quote_changed": "Rrrrrr, hah, is gewoon Boef man.test, jij bent vies maar ik doe gemener. In de club, komtest moeder tegen. En ik wil snel weg wantest we moeten wegen. En je klant is geholpen, je moetest vroeger wezen. Ik was alles kwijt, maar floetest herenigd. Voor me zondes af en toe gebeden. Ik gtest uit eten voor een goede prijs. Ik ben een uitgever, ze boeken mij. Van alarm voortest aan de achterkant. Dus ze komen via voor, maar wat dacht je dan?",
        "person": "Boef",
        "year": 2017,
        "test": "random toevoeging woorden",
    }
]

In [ ]:
for items, name in [(cruijff_score_inputs, 'cruijff'), (different_score_input, 'different')]:
    score_result_test_results = []
    for test in items:
        test["score"] = score_dutch_text(test["quote"], test["quote_changed"])
        test["changes"] = compare_strings(test["quote"], test["quote_changed"])
        
        score_result_test_results.append(test)
   
    path =  research_dir+name+'_score_test.json'
    # try: 
    #     os.makedirs(path)
    # except: 
    #     pass
    print(score_result_test_results)
    score_result_test_results.sort(key=lambda x: x["score"])
    with open(path, 'w') as f:
        json.dump(score_result_test_results, f, indent=4)

[{'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet schieten, nders kun je niet scoren.', 'person': 'Johan Cruijff', 'year': 1980, 'test': 'enkele letterverandering', 'score': 0.19370876948914964, 'changes': [('delete', (18, 19), '')]}, {'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet schoten, anders kun je niet scoren.', 'person': 'Johan Cruijff', 'year': 1980, 'test': 'enkele woordverandering, geen betekenisverandering', 'score': 0.21462842758854445, 'changes': [('replace', (11, 13), 'o')]}, {'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet roeien, anders kun je niet scoren.', 'person': 'Johan Cruijff', 'year': 1980, 'test': 'enkele woordverandering, wel betekenisverandering', 'score': 0.3282647912249081, 'changes': [('replace', (8, 11), 'roe'), ('delete', (12, 14), '')]}, {'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet schoten, anders kun je niet 

In [ ]:
test_images = [
    {
        "path": research_dir+''+'.png',
        "reason": "recht boven donker ondergrond",
    },
    {
        "path": research_dir+''+'.png',
        "reason": "recht boven licht ondergrond",
    },
    {
        "path": research_dir+''+'.png',
        "reason": "schuin donker onder",
    },
    {
        "path": research_dir+''+'.png',
        "reason": "gescanned normaal",
    },
    {
        "path": research_dir+''+'.png',
        "reason": "gescanned met moeilijke tekst en vraagherkenning",
    },
]

models = [
    {
        "provider": "openai",
        "model_name": "",
        "reason": ""
    },
    {
        "provider": "openai",
        "model_name": "",
        "reason": ""
    },
    {
        "provider": "openai",
        "model_name": "",
        "reason": ""
    },
    {
        "provider": "google",
        "model_name": "",
        "reason": ""
    },

]

temperatures = [
    {
        "temperature": 0,
        "repeat": 1,
    },
    {
        "temperature": 0.5,
        "repeat": 3,
    },
    {
        "temperature": 1,
        "repeat": 5,
    },
    {
        "temperature": 1.5,
        "repeat": 6,
    },
]

transcribe_texts = [
    {
        "text": "",
        "reason": ""
    },
    {
        "text": "",
        "reason": ""
    },
    {
        "text": "",
        "reason": ""
    },
    {
        "text": "",
        "reason": ""
    },
]



In [ ]:
from scan_module import scan_page

def scan_test_page(image,model,temperature, repeats, transcribe_text):
    pass